<a href="https://colab.research.google.com/github/bobzinin/python-internals-project/blob/numba/first_sight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

numba - библиотека, предназначенная для ускорения высоконагруженных участков кода.

Большую ее функциональность реализуют различные обёртки.
Наиболее популярная из них -- @jit

In [4]:
import os
import numpy as np
os.environ['NUMBA_DUMP_CFG'] = '1'
os.environ['NUMBA_DUMP_IR'] = '1'
from numba import jit

@jit
def JitHeavyFunction(x, y):
    ret = []
    for elemx in x:
        for elemy in y:
            ret.append(elemx**elemy + elemx*elemy)
            

def HeavyFunction(x, y):
    ret = []
    for elemx in x:
        for elemy in y:
            ret.append(elemx**elemy + elemx*elemy)
            
            
a = np.arange(1000)
b = np.linspace(0, 10, 100)

In [5]:
import dis

print(dis.dis(JitHeavyFunction))

  7           0 RESUME                   0

  9           2 BUILD_LIST               0
              4 STORE_FAST               2 (ret)

 10           6 LOAD_FAST                0 (x)
              8 GET_ITER
        >>   10 FOR_ITER                35 (to 84)
             14 STORE_FAST               3 (elemx)

 11          16 LOAD_FAST                1 (y)
             18 GET_ITER
        >>   20 FOR_ITER                28 (to 80)
             24 STORE_FAST               4 (elemy)

 12          26 LOAD_FAST                2 (ret)
             28 LOAD_ATTR                1 (NULL|self + append)
             48 LOAD_FAST                3 (elemx)
             50 LOAD_FAST                4 (elemy)
             52 BINARY_OP                8 (**)
             56 LOAD_FAST                3 (elemx)
             58 LOAD_FAST                4 (elemy)
             60 BINARY_OP                5 (*)
             64 BINARY_OP                0 (+)
             68 CALL                     1
           

# Ступени компиляции numba

Используя ключи дебага, можно отследить то, как numba оптимизирует python код

Всего выделяют 8 ключевых ступеней:

1. **Analyze bytecode** - numba получает bytecode, создает CFG (control flow graph) и Data flow graph
2. **Generate IR** - Переводит байт-код в более удобное внутреннее представление numba (intermediate representation)
3. **Rewrite IR** - Переписывает IR с учетом некоторых оптимизаций
4. **Infer types** - Статически типизирует переменные где возможно. Если тип не определяется, вызывается pyobject и происходит откат в object mode
5. **Rewrite typed IR** - Переписывает типизированное промежуточное представление
6. **Perform automatic parallelization** - Автоматическая параллелизация циклов (parfor)
7. **Generate LLVM IR** - Создание LLVM промежуточного представления
8. **Generate machine code** - Генерация нативного машинного кода

**Путь**: Python код → Bytecode → Numba IR → Typed IR → Optimized IR → LLVM IR → Machine Code

In [6]:
JitHeavyFunction(a, b)

CFG adjacency lists:
{0: [12], 12: [16, 86], 16: [22], 22: [26, 82], 26: [22], 82: [12], 86: []}
CFG dominators:
{0: {0},
 12: {0, 12},
 16: {16, 0, 12},
 22: {16, 0, 12, 22},
 26: {16, 0, 22, 26, 12},
 82: {16, 0, 82, 22, 12},
 86: {0, 12, 86}}
CFG post-dominators:
{0: {0, 12, 86},
 12: {12, 86},
 16: {16, 82, 86, 22, 12},
 22: {86, 82, 12, 22},
 26: {82, 86, 22, 26, 12},
 82: {82, 12, 86},
 86: {86}}
CFG back edges: [(26, 22), (82, 12)]
CFG loops:
{12: Loop(entries={0}, exits={86}, header=12, body={12, 16, 82, 22, 26}),
 22: Loop(entries={16}, exits={82}, header=22, body={26, 22})}
CFG node-to-loops:
{0: [], 12: [12], 16: [12], 22: [22, 12], 26: [22, 12], 82: [12], 86: []}
CFG backbone:
{0, 12, 86}
---------------------------IR DUMP: JitHeavyFunction----------------------------
label 0:
    x = arg(0, name=x)                       ['x']
    y = arg(1, name=y)                       ['y']
    ret = build_list(items=[])               ['ret']
    $10get_iter.2 = getiter(value=x)         

## @jit args
nopython - будет стараться не прибегать к использованию python кода, а как можно больше генерировать собственный машинный код

cache - закэширует машинный код функции

nogil - для многопоточных приложений

parallel - попытается автоматически распараллелить циклы внутри функции

inline - для использования функции как inline





Одна из вещей, которая даёт значительный прирост производительности --- статическая типизация значений, принимаемых функцией. Вот как можно помочь numba с определением типов:
```python
@decorator([return_type(input_type_1, input_type_2…)])
```

In [ ]:
from numba import float64, int32

@jit([float64(float64, float64)]) # функция принимающая 2 float значения и возвращающая другой float
def add(x, y):
    return x + y


@jit([int32(int32, int32), float64(float64, float64)]) # если хотим добавить несколько вариантов
def multiply(x, y):
    return x * y

@jit([float64(float64[:])])
def sum(a):
    s = 0
    for elem in a:
        s += elem
    return s

NameError: name 'float64' is not defined


## @vectorize
векторизация -- процесс, позволяющий за один "такт" обрабатывать по несколько элементов

в нашем случае numba создаёт numpy.ufunc(universal function), которая позволяет быстрее выполнять одинаковые
операции над элементами массива

ufunc быстрее и удобнее за счёт того, что:
- В numpy есть определенные договорённости по работе с многомерными массивами, которые позволяют использовать оптимизированные функции с другими совместимыми
- numba всё равно частично компиллирует нашу функцию в машинный код, поэтому он выполняется быстрее

In [ ]:
from numba import vectorize

@vectorize([float64(float64, float64)])
def add(x, y):
    return x + y

## @stencil
позволяет удобно работать с функциями, зависящими от окружения элемента(фильтры на фотографиях/сглаживания и т.п.)

это улучшает читаемость кода

ну и как же без оптимизации с помощью машинного кода

In [ ]:
from numba import stencil

@stencil
def smooth_1d(a):
    return (a[-1] + a[0] + a[1]) / 3.0